In [1]:
%load_ext autoreload
%autoreload 2

# Parameter Estimation
In this notebook I attempt to approach the designation of parameters anew. The main goal is to make clear the source of all parameters, be it from scientific article, or from a computation. The purpose is to make the research more accessible/understandable, as well as ensuring correctness of parameters following a parameter change.

In [2]:
SHARED_PARAMS = {
    "bbb": [
        ("arteries", "pvs_arteries"),
        ("capillaries", "pvs_capillaries"),
        ("veins", "pvs_veins")
    ],
    "aef": [
        ("pvs_arteries", "ecs"),
        ("pvs_capillaries", "ecs"),
        ("pvs_veins", "ecs")
    ],
    "pvs": ["pvs_arteries", "pvs_capillaries", "pvs_veins"],
    "csf": ["ecs", "pvs_arteries", "pvs_capillaries", "pvs_veins"],
    "blood": ["arteries", "capillaries", "veins"],
    "all": ["ecs", "pvs_arteries", "pvs_capillaries", "pvs_veins", "arteries", "capillaries", "veins"]
}

In [3]:
def retrieve_aliases(param):
    split = param.split("-")
    return [word for word in split[1:] if word in SHARED_PARAMS]


def unpack_shared_parameters(parameters):
    newparameters = {}
    for param, value in parameters.items():
        aliases = retrieve_aliases(param)
        for alias in aliases:
            if type(SHARED_PARAMS[alias][0]) == str:
                newnames = [param.replace(alias, key) for key in SHARED_PARAMS[alias]]
            else:
                raise 
                newnames = [param.replace(alias, "-".join(key)) for key in SHARED_PARAMS[alias]]
            
            for name in newnames:
                newparameters[name] = value
        
        if len(aliases) == 0:
            newparameters[param] = value
        
    return newparameters

In [4]:
from pint import UnitRegistry
from pint import Quantity
from multirat.parameters import unpack_shared_parameters

ureg = UnitRegistry()

mmHg = ureg.mmHg
Pa = ureg.Pa
m = ureg.m
mm = ureg.mm
nm = ureg.nm
s = ureg.s
minute = ureg.min
mL = ureg.mL

In [5]:
base_parameters = {
    # Brain Volume
    "brain_volume": 2450. * mm**3,
    "human_brain_volume": 1.0e6 * mm**3,
    # Pressure boundary parameters
    "pressure_boundary-pvs_arteries": 4.74 * mmHg,
    "pressure_boundary-ecs": 3.74 * mmHg,
    "pressure_boundary-pvs_veins": 3.36 * mmHg,
    "pressure_boundary-arteries": 120. * mmHg,
    "pressure_boundary-veins": 7.0 * mmHg,
    "diffusion_free_inulin-all": 2.98 * mm**2 / s,
    "tortuosity-all": 1.7,
    "osmotic_pressure-blood": 20. * mmHg,
    "permeability_inulin-bbb": 0.0 * mm / s,  # Does not cross the BBB
    "porosity-ecs": 0.14,
    "vasculature_volume_fraction": 0.0329,
    "vasculature_fraction-arteries": 0.2,  #  (Lee et al. 2001)
    "vasculature_fraction-capillaries": 0.1, # (Lee et al. 2001)
    "vasculature_fraction-veins": 0.7,  # (Lee et al. 2001)
    "pvs_volume_fraction":  0.003,  # (Ballerini, et al. 2020)
    "viscosity-blood": 2.67e-3 * Pa*s,  #  (Guo et al. 2019)
    "viscosity-csf": 7.0e-4 * Pa*s,  #  (Bloomfield, Johnston, Bilston 1998)
    "permeability-ecs": 2.0e-11 * mm**2,
    "resistance-ecs": 4.56 * Pa * s / mm**2,
    # Compute more permeabilities
    "hydraulic_conductivity-ecs-arteries": 9.1e-10 * mm / (Pa * s),
    "hydraulic_conductivity-ecs-capillaries": 1.0e-10 * mm / (Pa * s),
    "hydraulic_conductivity-ecs-veins": 2.0e-11 * mm / (Pa * s),
    "hydraulic_conductivity-capillaries-pvs_capillaries": 2.25e-15 * m**2 *(Pa * s),  # (Asgari 2015 - How astrocyte networks may contribute...)
    "surface_volume_ratio-ecs-arteries": 3. * 1 / mm,   # (Assumed 1/3 of capillary)
    "surface_volume_ratio-ecs-capillaries": 9. * 1 / mm,  # (Smith and Humphrey 2007)
    "surface_volume_ratio-ecs-veins": 3. * 1 / mm,  # (Assumed 1/3 of capillary)
    "flowrate_blood": 700 * mm**3 / s,
    "pressure_drop-arteries-capillaries": 50.0 * mmHg,
    "pressure_drop-capillaries-veins": 10.0 * mmHg,
    "flowrate_csf": 3.33 * mm**3 / s,
    "pressure_drop-pvs_arteries-pvs_capillaries": 1.0 * mmHg,
    "pressure_drop-pvs_capillaries-pvs_veins": 0.25 * mmHg,
    "resistance_network-pvs_arteries": 1.14 * mmHg / (mL / minute),
    "resistance_network-pvs_veins": 1.75e-3 * mmHg / (mL / minute),
    "resistance_network-pvs_capillaries": 32.4 * mmHg / (mL / minute),
    "resistance_network-ecs": 0.57 * mmHg / (mL / minute),
    "resistance_interface-ecs-arteries": 0.57 * mmHg / (mL / minute),
    "resistance_interface-ecs-veins": 0.64 * mmHg / (mL / minute),
    "resistance_interface-ecs-capillaries": 125.31 * mmHg / (mL / minute),
    "thickness_aef_membrane": 1000.0 * nm,  # (Fu, 2010 - Model for BBB permeability to water and small solutes)
    "aef_cleft_distance": 2.5 * nm, # (Fu, 2010) Min value
#     "aef_cleft_distance": 2000.0 * nm,  # Max value
    
    
    
    # ###### 
    "osmotic_pressure-blood": 20 * mmHg,
    "osmotic_pressure_fraction-csf": 0.2,  # Osm. press. computed as this constat * osmotic_pressure-blood
    "reflection_inulin-all-all": 0.2,
}
shared_parameters = unpack_shared_parameters(base_parameters)

def print_quantities(p):
    for key, value in p.items():
        print(f"{key:<55}: {value}")
        

print_quantities(base_parameters)

brain_volume                                           : 2450.0 millimeter ** 3
human_brain_volume                                     : 1000000.0 millimeter ** 3
pressure_boundary-pvs_arteries                         : 4.74 millimeter_Hg
pressure_boundary-ecs                                  : 3.74 millimeter_Hg
pressure_boundary-pvs_veins                            : 3.36 millimeter_Hg
pressure_boundary-arteries                             : 120.0 millimeter_Hg
pressure_boundary-veins                                : 7.0 millimeter_Hg
diffusion_free_inulin-all                              : 2.98 millimeter ** 2 / second
tortuosity-all                                         : 1.7
osmotic_pressure-blood                                 : 20 millimeter_Hg
permeability_inulin-bbb                                : 0.0 millimeter / second
porosity-ecs                                           : 0.14
vasculature_volume_fraction                            : 0.0329
vasculature_fraction-arterie

In [6]:
print_quantities(shared_parameters)

brain_volume                                           : 2450.0 millimeter ** 3
human_brain_volume                                     : 1000000.0 millimeter ** 3
pressure_boundary-pvs_arteries                         : 4.74 millimeter_Hg
pressure_boundary-ecs                                  : 3.74 millimeter_Hg
pressure_boundary-pvs_veins                            : 3.36 millimeter_Hg
pressure_boundary-arteries                             : 120.0 millimeter_Hg
pressure_boundary-veins                                : 7.0 millimeter_Hg
diffusion_free_inulin-ecs                              : 2.98 millimeter ** 2 / second
diffusion_free_inulin-pvs_arteries                     : 2.98 millimeter ** 2 / second
diffusion_free_inulin-pvs_capillaries                  : 2.98 millimeter ** 2 / second
diffusion_free_inulin-pvs_veins                        : 2.98 millimeter ** 2 / second
diffusion_free_inulin-arteries                         : 2.98 millimeter ** 2 / second
diffusion_free_inulin-

In [7]:
def effective_diffusion(D_free, tortuosity):
    return D_free / tortuosity**2


def compute_effective_diffusion(p):
    return {f"diffusion_eff_inulin-{x}": 
            effective_diffusion(
                p[f"diffusion_free_inulin-{x}"],
                p[f"tortuosity-{x}"]
            )
        for x in SHARED_PARAMS["all"]
    }


def subnetwork_porosity(network_volume_fraction, subnetwork_fraction):
    return network_volume_fraction * subnetwork_fraction


def compute_vasculature_porosity(p):
    return {
        f"porosity-{x}":
            subnetwork_porosity(
                p['vasculature_volume_fraction'],
                p[f'vasculature_fraction-{x}']
            )
        for x in SHARED_PARAMS["blood"]
    }


def compute_pvs_porosity(p):
    return {
        f"porosity-{x}":
            subnetwork_porosity(
                p['pvs_volume_fraction'],
                p[f'vasculature_fraction-{x.replace("pvs_", "")}']
            )
        for x in SHARED_PARAMS["pvs"]
    }

def compute_porosities(p):
    return {**compute_vasculature_porosity(p), **compute_pvs_porosity(p)}


def permeability(viscosity, resistance, length_area_ratio):
    return viscosity * length_area_ratio / resistance


def compute_permeabilities(p):
    return {
        f"permeability-{x}": 
            permeability(
                p[f'viscosity-{x}'],
                p[f"resistance_network-{x}"],
                p[f"length_area_ratio"]
            )
        for x in [*SHARED_PARAMS["csf"]] #, *SHARED_PARAMS["blood"]]
    }

def compute_length_area_ratio(p):
    kappa = p["permeability-ecs"]
    R = p["resistance-ecs"]
    mu = p["resistance-ecs"]
    return {"length_area_ratio": kappa * R / mu}

def compute_parameters(base):
    """TODO: Add unit converter already here!"""
    out = unpack_shared_parameters(base)
    out = {**compute_effective_diffusion(out), **out}
    out = {**compute_porosities(out), **out}
    out = {**compute_length_area_ratio(out), **out}
    out = {**compute_permeabilities(out), **out}
    return out

In [11]:
def compute_partial_fluid_transfer(brain_volume, resistance, total_transfer):
    new_resistance = 1.0 / (total_transfer * brain_volume) - resistance
    return 1.0 / (new_resistance * brain_volume)


def compute_connected_fluid_transfer(brain_volume, flow_rate, pressure_drop):
    return flow_rate / (pressure_drop * brain_volume)


def compute_convective_fluid_transfer(p):
    """There are several ways to compute these transfer coefficients. Currently
    there seeems to be an issue wrt. the transfer between different compartments
    at the capillary level: If we use the given formula to compute transfer between
    ecs and pericapillary space, why do we need an alternative between capillary and
    venuous? Currently we are using the same formyla for all, but I suspect there is
    an error to be found here.
    """
    # FIXME: replace potential error in capillaries/paracapillary resistance.
    ecs_to_blood = {
        f"convective_fluid_transfer-ecs-{x}":
            p[f"hydraulic_conductivity-ecs-{x}"] * p[f"surface_volume_ratio-ecs-{x}"]
        for x in SHARED_PARAMS["blood"]
    }
    ecs_to_pvs = {
        f"convective_fluid_transfer-ecs-{x}":
            1. / (p["human_brain_volume"] * p[f"resistance_interface-ecs-{x.replace('pvs_', '')}"])
        for x in SHARED_PARAMS["pvs"]
    }
    pvs_to_blood = {
        f"convective_fluid_transfer-{x}-pvs_{x}":
            compute_partial_fluid_transfer(
                p["human_brain_volume"],
                p[f"resistance_interface-ecs-{x}"],
                ecs_to_blood[f"convective_fluid_transfer-ecs-{x}"]
            )
        for x in SHARED_PARAMS["blood"]
    }
    connected_blood_transfer = {
        f"convective_fluid_transfer-{i}-{j}":
            compute_connected_fluid_transfer(
                p["human_brain_volume"],
                p[f"flowrate_blood"],
                p[f"pressure_drop-{i}-{j}"]
            )
        for (i, j) in [
            ("arteries", "capillaries"),
            ("capillaries", "veins"),
        ]
    }
    connected_csf_transfer = {
        f"convective_fluid_transfer-{i}-{j}":
            compute_connected_fluid_transfer(
                p["human_brain_volume"],
                p[f"flowrate_csf"],
                p[f"pressure_drop-{i}-{j}"]
            )
        for (i, j) in [
            ("pvs_arteries", "pvs_capillaries"),
            ("pvs_capillaries", "pvs_veins"),
        ]
    }
    convective_transfer = {
        **ecs_to_pvs,
        **pvs_to_blood,
        **connected_blood_transfer,
        **connected_csf_transfer,
    }
    return {
        key: val.to(1/(Pa * s)) for key, val in convective_transfer.items()
    }

p = compute_convective_fluid_transfer(shared_parameters)
print_quantities(p)

convective_fluid_transfer-ecs-pvs_arteries             : 2.1931625024726804e-07 / pascal / second
convective_fluid_transfer-ecs-pvs_capillaries          : 9.976080332051935e-10 / pascal / second
convective_fluid_transfer-ecs-pvs_veins                : 1.953285353764731e-07 / pascal / second
convective_fluid_transfer-arteries-pvs_arteries        : 2.7644107716509763e-09 / pascal / second
convective_fluid_transfer-capillaries-pvs_capillaries  : 9.19849729988111e-09 / pascal / second
convective_fluid_transfer-veins-pvs_veins              : 6.001843614995655e-11 / pascal / second
convective_fluid_transfer-arteries-capillaries         : 1.0500862061839191e-07 / pascal / second
convective_fluid_transfer-capillaries-veins            : 5.250431030919595e-07 / pascal / second
convective_fluid_transfer-pvs_arteries-pvs_capillaries : 2.4977050475660363e-08 / pascal / second
convective_fluid_transfer-pvs_capillaries-pvs_veins    : 9.990820190264145e-08 / pascal / second


## Graveyard